In [2]:
import matplotlib.pyplot as plt
from coffea.util import load
from pocket_coffea.utils.plot_utils import Shape
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_png { height: auto; }</style>"))
import pickle
import numpy as np
import os
if not os.path.exists('hists'):
    os.makedirs('hists')
import awkward as ak
from omegaconf import OmegaConf
import math
import mplhep as hep

In [3]:
o = load("out_columns/output_allNoDupes.coffea")
#o2 = load("output_newconfig11_TTbbDiLeptonic_Powheg_2018.coffea")
o2 = load("output_ttbb_all.coffea")
parameters_dump = OmegaConf.load("/afs/cern.ch/user/r/rmccarth/public/plotting_style.yaml")

In [21]:
print(o['columns']['ZJetsToQQ_HT600to800__nominal']['ZJetsToQQ_HT600to800_v7__2018']['baseline'].keys())
print(o['sum_genweights'].keys())
print(o['columns'].keys())
print(o2['columns'].keys())

dict_keys(['weight_nominal', 'weight_pileupUp', 'weight_pileupDown', 'weight_sf_ele_recoUp', 'weight_sf_ele_recoDown', 'weight_sf_ele_idUp', 'weight_sf_ele_idDown', 'weight_sf_mu_idUp', 'weight_sf_mu_idDown', 'weight_sf_mu_isoUp', 'weight_sf_mu_isoDown', 'weight_sf_jet_puIdUp', 'weight_sf_jet_puIdDown', 'weight_sf_btag_hfUp', 'weight_sf_btag_hfDown', 'weight_sf_btag_lfUp', 'weight_sf_btag_lfDown', 'weight_sf_btag_hfstats1Up', 'weight_sf_btag_hfstats1Down', 'weight_sf_btag_hfstats2Up', 'weight_sf_btag_hfstats2Down', 'weight_sf_btag_lfstats1Up', 'weight_sf_btag_lfstats1Down', 'weight_sf_btag_lfstats2Up', 'weight_sf_btag_lfstats2Down', 'weight_sf_btag_cferr1Up', 'weight_sf_btag_cferr1Down', 'weight_sf_btag_cferr2Up', 'weight_sf_btag_cferr2Down', 'JetGood_N', 'JetGood_eta', 'JetGood_pt', 'JetGood_phi', 'JetGood_btagDeepFlavB', 'JetGood_genJetIdx', 'FatJetGood_N', 'FatJetGood_eta', 'FatJetGood_pt', 'FatJetGood_phi', 'FatJetGood_mass', 'FatJetGood_msoftdrop', 'FatJetGood_tau1', 'FatJetGood_t

In [4]:
def getName(sam_name):
    sam_namey = sam_name+'__2018'
    if 'ttHToNonbb' in sam_name: sam_namey ='ttHToNonbb_Powheg_2018'
    if 'DYJetsToLL_M-50' in sam_name: sam_namey ='DYJetsToLL_M-50_v7__2018'
    if 'ZJetsToQQ' in sam_name: sam_namey = sam_name + '_v7__2018'
    if 'ST' in sam_name: sam_namey = sam_name + '_v7__2018'
    if 'QCD' in sam_name: sam_namey = sam_name + '_v7__2018'
    if 'WJets' in sam_name: sam_namey = sam_name + '_v7__2018' 
    if 'WJetsToLNu' in sam_name: sam_namey = sam_name + '__2018'
    if 'TTWJets' in sam_name: sam_namey = sam_name + '__2018'
    if 'TTGJets' in sam_name: sam_namey = sam_name + '__2018'
    if 'THW' in sam_name: sam_namey = sam_name + '__2018'
    if 'WW' in sam_name: sam_namey = sam_name + '__2018'
    if 'WZ' in sam_name: sam_namey = sam_name + '__2018'
    if 'ZZ' in sam_name: sam_namey = sam_name + '__2018'
    if 'ttHTobb' in sam_name: sam_namey = sam_name + '_2018'
    return sam_namey

def getPlotParams(var):
    #return format: bins, xmin, xmax, label
    col = var.split("_")[0]
    quantity = var.split("_")[1]
    if quantity == 'eta':
        return 30, -2.5, 2.5, fr"{col} $\eta$"
    if quantity == 'phi':
        return 32, -math.pi, math.pi, fr"{col} $\phi$"
    if ("FatJet" in col) and not ("N" in quantity) and not ("Flavour" in quantity):
        if quantity == 'pt':
            return 100, 0, 1000, r"FatJet $p_{T}$ [GeV]"
        elif quantity == 'mass':
            return 50, 0, 400, r"FatJet mass [GeV]"
        elif quantity == 'msoftdrop':
            return 50, 0, 400, r"FatJet $m_{SD}$ [GeV]"
        elif quantity == 'rho':
            return 100, -8, 0, r"FatJet $\rho$"
        elif quantity == 'rhoQCD':
            return 100, -3, 0.5, r"FatJet $\rho$QCD"
        elif quantity == 'tau1':
            return 20, 0, 1, r"$\tau_{1}$"
        elif quantity == 'tau2':
            return 20, 0, 1, r"$\tau_{2}$"
        elif quantity == 'tau3':
            return 20, 0, 1, r"$\tau_{3}$"
        elif quantity == 'tau4':
            return 20, 0, 1, r"$\tau_{4}$"
        else:
            return 20, 0, 1, f"{col}"
    elif "Jet" in col and not ("N" in quantity)and not ("Flavour" in quantity):
        if quantity == 'pt':
            return 100, 0, 1000, r"Jet $p_{T}$ [GeV]"
        elif quantity == 'btagDeepFlavB':
            return 30, 0, 1, "AK4 DeepJet b-tag score"
        elif quantity == 'mass':
            return 50, 0, 400, r"Jet mass [GeV]"
    else:
        return -1, -1, -1, ""
    
def deltaR(phis,etas):
    leftPhi,rightPhi = ak.unzip(phis)
    leftEta,rightEta = ak.unzip(etas)
    return np.sqrt(pow(leftEta-rightEta,2)+pow(leftPhi-rightPhi,2))


In [25]:
#Calculate and append new branches for plotting
for sample in o['columns'].keys():
    for year in o['columns'][sample].keys():
        for cat in o['columns'][sample][year].keys():
            print(sample,year,cat)
            leptonsPhi = ak.from_numpy(o['columns'][sample][year][cat]['LeptonGood_phi'].value)
            leptonsEta = ak.from_numpy(o['columns'][sample][year][cat]['LeptonGood_eta'].value)
            leptonsN = ak.from_numpy(o['columns'][sample][year][cat]['LeptonGood_N'].value)
            leptonsPhi = ak.unflatten(leptonsPhi,leptonsN)
            leptonsEta = ak.unflatten(leptonsEta,leptonsN) 
            
            jetPhi = ak.from_numpy(o['columns'][sample][year][cat]['JetGood_phi'].value)
            jetEta = ak.from_numpy(o['columns'][sample][year][cat]['JetGood_eta'].value)
            jetN = ak.from_numpy(o['columns'][sample][year][cat]['JetGood_N'].value)
            jetPhi = ak.unflatten(jetPhi,jetN)
            jetEta = ak.unflatten(jetEta,jetN)
            
            fatJetPhi = ak.from_numpy(o['columns'][sample][year][cat]['FatJetGood_phi'].value)
            fatJetEta = ak.from_numpy(o['columns'][sample][year][cat]['FatJetGood_eta'].value)
            fatJetN = ak.from_numpy(o['columns'][sample][year][cat]['FatJetGood_N'].value)
            fatJetPhi = ak.unflatten(fatJetPhi,fatJetN)
            fatJetEta = ak.unflatten(fatJetEta,fatJetN)
            
            bbfatJetLTau1 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodL_tau1'].value)
            bbfatJetLTau2 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodL_tau2'].value)
            bbfatJetLTau3 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodL_tau3'].value)
            bbfatJetLTau4 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodL_tau4'].value)
            o['columns'][sample][year][cat]['BBFatJetGoodL_tau21'] = np.divide(bbfatJetLTau2,bbfatJetLTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodL_tau32'] = np.divide(bbfatJetLTau3,bbfatJetLTau2)
            o['columns'][sample][year][cat]['BBFatJetGoodL_tau31'] = np.divide(bbfatJetLTau3,bbfatJetLTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodL_tau43'] = np.divide(bbfatJetLTau4,bbfatJetLTau3)
            
            bbfatJetMTau1 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodM_tau1'].value)
            bbfatJetMTau2 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodM_tau2'].value)
            bbfatJetMTau3 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodM_tau3'].value)
            bbfatJetMTau4 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodM_tau4'].value)
            o['columns'][sample][year][cat]['BBFatJetGoodM_tau21'] = np.divide(bbfatJetMTau2,bbfatJetMTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodM_tau32'] = np.divide(bbfatJetMTau3,bbfatJetMTau2)
            o['columns'][sample][year][cat]['BBFatJetGoodM_tau31'] = np.divide(bbfatJetMTau3,bbfatJetMTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodM_tau43'] = np.divide(bbfatJetMTau4,bbfatJetMTau3)
            
            bbfatJetTTau1 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodT_tau1'].value)
            bbfatJetTTau2 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodT_tau2'].value)
            bbfatJetTTau3 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodT_tau3'].value)
            bbfatJetTTau4 = ak.from_numpy(o['columns'][sample][year][cat]['BBFatJetGoodT_tau4'].value)
            o['columns'][sample][year][cat]['BBFatJetGoodT_tau21'] = np.divide(bbfatJetTTau2,bbfatJetTTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodT_tau32'] = np.divide(bbfatJetTTau3,bbfatJetTTau2)
            o['columns'][sample][year][cat]['BBFatJetGoodT_tau31'] = np.divide(bbfatJetTTau3,bbfatJetTTau1)
            o['columns'][sample][year][cat]['BBFatJetGoodT_tau43'] = np.divide(bbfatJetTTau4,bbfatJetTTau3)

            jetLeptonEta = ak.cartesian([leptonsEta,jetEta],axis=1)
            jetLeptonPhi = ak.cartesian([leptonsPhi,jetPhi],axis=1)
            jetFatJetEta = ak.cartesian([fatJetEta,jetEta],axis=1)
            jetFatJetPhi = ak.cartesian([fatJetPhi,jetPhi],axis=1)
            fatjetLeptonEta = ak.cartesian([leptonsEta,fatJetEta],axis=1)
            fatjetLeptonPhi = ak.cartesian([leptonsPhi,fatJetPhi],axis=1)
            leptonsPhi = ak.to_list(ak.combinations(leptonsPhi,2))
            leptonsEta = ak.to_list(ak.combinations(leptonsEta,2))
            jetPhi = ak.to_list(ak.combinations(jetPhi,2))
            jetEta = ak.to_list(ak.combinations(jetEta,2))
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0):
                leptonsDeltaR = deltaR(leptonsPhi,leptonsEta)
                leptonsDeltaR_N = ak.count(leptonsDeltaR,axis=1)
                leptonsDeltaR = ak.flatten(leptonsDeltaR,axis=1)
            else:
                leptonsDeltaR = []
                leptonsDeltaR_N = ak.zeros_like(leptonsN)
            o['columns'][sample][year][cat]['LeptonGoodCombinations_deltaR'] = leptonsDeltaR
            o['columns'][sample][year][cat]['LeptonGoodCombinations_N'] = leptonsDeltaR_N
            
            if(len(jetPhi)!=0 and len(jetEta)!=0):
                jetDeltaR = deltaR(jetPhi,jetEta)
                jetDeltaR_N = ak.count(jetDeltaR,axis=1)
                jetDeltaR = ak.flatten(jetDeltaR,axis=1)
            else:
                jetDeltaR = []
                jetDeltaR_N = ak.zeros_like(jetN)
            o['columns'][sample][year][cat]['JetGoodCombinations_deltaR'] = jetDeltaR
            o['columns'][sample][year][cat]['JetGoodCombinations_N'] = jetDeltaR_N
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0 and len(jetPhi)!=0 and len(jetEta)!=0):
                jetLeptonDeltaR = deltaR(jetLeptonPhi,jetLeptonEta)
                jetLeptonDeltaR_N = ak.count(jetLeptonDeltaR,axis=1)
                jetLeptonDeltaR = ak.flatten(jetLeptonDeltaR,axis=1)
            else:
                jetLeptonDeltaR = []
                jetLeptonDeltaR_N = ak.zeros_like(leptonsN)
            o['columns'][sample][year][cat]['JetLeptonGoodCombinations_deltaR'] = jetLeptonDeltaR
            o['columns'][sample][year][cat]['JetLeptonGoodCombinations_N'] = jetLeptonDeltaR_N
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0 and len(fatJetPhi)!=0 and len(fatJetEta)!=0):
                fatjetLeptonDeltaR = deltaR(fatjetLeptonPhi,fatjetLeptonEta)
                fatjetLeptonDeltaR_N = ak.count(fatjetLeptonDeltaR,axis=1)
                fatjetLeptonDeltaR = ak.flatten(fatjetLeptonDeltaR,axis=1)
            else:
                fatjetLeptonDeltaR = []
                fatjetLeptonDeltaR_N = ak.zeros_like(leptonsN)
            o['columns'][sample][year][cat]['FatJetLeptonGoodCombinations_deltaR'] = fatjetLeptonDeltaR
            o['columns'][sample][year][cat]['FatJetLeptonGoodCombinations_N'] = fatjetLeptonDeltaR_N
            
            if(len(fatJetPhi)!=0 and len(fatJetEta)!=0 and len(jetPhi)!=0 and len(jetEta)!=0):
                jetFatJetDeltaR = deltaR(jetFatJetPhi,jetFatJetEta)
                jetFatJetDeltaR_N = ak.count(jetFatJetDeltaR,axis=1)
                jetFatJetDeltaR = ak.flatten(jetFatJetDeltaR,axis=1)
            else:
                jetFatJetDeltaR = []
                jetFatJetDeltaR_N = ak.zeros_like(leptonsN)
            o['columns'][sample][year][cat]['JetFatJetGoodCombinations_deltaR'] = jetFatJetDeltaR
            o['columns'][sample][year][cat]['JetFatJetGoodCombinations_N'] = jetFatJetDeltaR_N

QCD_HT500to700__nominal QCD_HT500to700_v7__2018 baseline
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 ee
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 emu
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 mumu
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 baseline
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 ee
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 emu
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 mumu
QCD_HT1500to2000__nominal QCD_HT1500to2000_v7__2018 baseline
QCD_HT1500to2000__nominal QCD_HT1500to2000_v7__2018 ee
QCD_HT1500to2000__nominal QCD_HT1500to2000_v7__2018 emu
QCD_HT1500to2000__nominal QCD_HT1500to2000_v7__2018 mumu
ST_s-channel_4f_leptonDecays__nominal ST_s-channel_4f_leptonDecays_v7__2018 baseline
ST_s-channel_4f_leptonDecays__nominal ST_s-channel_4f_leptonDecays_v7__2018 ee
ST_s-channel_4f_leptonDecays__nominal ST_s-channel_4f_leptonDecays_v7__2018 emu
ST_s-channel_4f_leptonDecays__nominal ST_s-channe

/usr/local/lib/python3.8/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in true_divide
  result = getattr(ufunc, method)(


TTGJets__nominal TTGJets__2018 ee
TTGJets__nominal TTGJets__2018 emu
TTGJets__nominal TTGJets__2018 mumu
THW__nominal THW__2018 baseline
THW__nominal THW__2018 ee
THW__nominal THW__2018 emu
THW__nominal THW__2018 mumu
WW__nominal WW__2018 baseline
WW__nominal WW__2018 ee
WW__nominal WW__2018 emu
WW__nominal WW__2018 mumu
WZ__nominal WZ__2018 baseline
WZ__nominal WZ__2018 ee
WZ__nominal WZ__2018 emu
WZ__nominal WZ__2018 mumu
ZZ__nominal ZZ__2018 baseline
ZZ__nominal ZZ__2018 ee
ZZ__nominal ZZ__2018 emu
ZZ__nominal ZZ__2018 mumu
WJetsToLNu_HT-800To1200__nominal WJetsToLNu_HT-800To1200__2018 baseline
WJetsToLNu_HT-800To1200__nominal WJetsToLNu_HT-800To1200__2018 ee
WJetsToLNu_HT-800To1200__nominal WJetsToLNu_HT-800To1200__2018 emu
WJetsToLNu_HT-800To1200__nominal WJetsToLNu_HT-800To1200__2018 mumu
QCD_HT2000toInf__nominal QCD_HT2000toInf_v7__2018 baseline
QCD_HT2000toInf__nominal QCD_HT2000toInf_v7__2018 ee
QCD_HT2000toInf__nominal QCD_HT2000toInf_v7__2018 emu
QCD_HT2000toInf__nominal QCD

In [27]:
#Calculate and append new branches for plotting
for sample in o2['columns'].keys():
    for year in o2['columns'][sample].keys():
        for cat in o2['columns'][sample][year].keys():
            print(sample,year,cat)
            leptonsPhi = ak.from_numpy(o2['columns'][sample][year][cat]['LeptonGood_phi'].value)
            leptonsEta = ak.from_numpy(o2['columns'][sample][year][cat]['LeptonGood_eta'].value)
            leptonsN = ak.from_numpy(o2['columns'][sample][year][cat]['LeptonGood_N'].value)
            leptonsPhi = ak.unflatten(leptonsPhi,leptonsN)
            leptonsEta = ak.unflatten(leptonsEta,leptonsN) 
            
            jetPhi = ak.from_numpy(o2['columns'][sample][year][cat]['JetGood_phi'].value)
            jetEta = ak.from_numpy(o2['columns'][sample][year][cat]['JetGood_eta'].value)
            jetN = ak.from_numpy(o2['columns'][sample][year][cat]['JetGood_N'].value)
            jetPhi = ak.unflatten(jetPhi,jetN)
            jetEta = ak.unflatten(jetEta,jetN)
            
            fatJetPhi = ak.from_numpy(o2['columns'][sample][year][cat]['FatJetGood_phi'].value)
            fatJetEta = ak.from_numpy(o2['columns'][sample][year][cat]['FatJetGood_eta'].value)
            fatJetN = ak.from_numpy(o2['columns'][sample][year][cat]['FatJetGood_N'].value)
            fatJetPhi = ak.unflatten(fatJetPhi,fatJetN)
            fatJetEta = ak.unflatten(fatJetEta,fatJetN)
            
            bbfatJetLTau1 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodL_tau1'].value)
            bbfatJetLTau2 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodL_tau2'].value)
            bbfatJetLTau3 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodL_tau3'].value)
            bbfatJetLTau4 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodL_tau4'].value)
            print(type())
            o2['columns'][sample][year][cat]['BBFatJetGoodL_tau21'] = np.divide(bbfatJetLTau2,bbfatJetLTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodL_tau32'] = np.divide(bbfatJetLTau3,bbfatJetLTau2)
            o2['columns'][sample][year][cat]['BBFatJetGoodL_tau31'] = np.divide(bbfatJetLTau3,bbfatJetLTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodL_tau43'] = np.divide(bbfatJetLTau4,bbfatJetLTau3)
            
            bbfatJetMTau1 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodM_tau1'].value)
            bbfatJetMTau2 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodM_tau2'].value)
            bbfatJetMTau3 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodM_tau3'].value)
            bbfatJetMTau4 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodM_tau4'].value)
            o2['columns'][sample][year][cat]['BBFatJetGoodM_tau21'] = np.divide(bbfatJetMTau2,bbfatJetMTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodM_tau32'] = np.divide(bbfatJetMTau3,bbfatJetMTau2)
            o2['columns'][sample][year][cat]['BBFatJetGoodM_tau31'] = np.divide(bbfatJetMTau3,bbfatJetMTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodM_tau43'] = np.divide(bbfatJetMTau4,bbfatJetMTau3)
            
            bbfatJetTTau1 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodT_tau1'].value)
            bbfatJetTTau2 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodT_tau2'].value)
            bbfatJetTTau3 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodT_tau3'].value)
            bbfatJetTTau4 = ak.from_numpy(o2['columns'][sample][year][cat]['BBFatJetGoodT_tau4'].value)
            o2['columns'][sample][year][cat]['BBFatJetGoodT_tau21'] = np.divide(bbfatJetTTau2,bbfatJetTTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodT_tau32'] = np.divide(bbfatJetTTau3,bbfatJetTTau2)
            o2['columns'][sample][year][cat]['BBFatJetGoodT_tau31'] = np.divide(bbfatJetTTau3,bbfatJetTTau1)
            o2['columns'][sample][year][cat]['BBFatJetGoodT_tau43'] = np.divide(bbfatJetTTau4,bbfatJetTTau3)

            jetLeptonEta = ak.cartesian([leptonsEta,jetEta],axis=1)
            jetLeptonPhi = ak.cartesian([leptonsPhi,jetPhi],axis=1)
            jetFatJetEta = ak.cartesian([fatJetEta,jetEta],axis=1)
            jetFatJetPhi = ak.cartesian([fatJetPhi,jetPhi],axis=1)
            fatjetLeptonEta = ak.cartesian([leptonsEta,fatJetEta],axis=1)
            fatjetLeptonPhi = ak.cartesian([leptonsPhi,fatJetPhi],axis=1)
            leptonsPhi = ak.to_list(ak.combinations(leptonsPhi,2))
            leptonsEta = ak.to_list(ak.combinations(leptonsEta,2))
            jetPhi = ak.to_list(ak.combinations(jetPhi,2))
            jetEta = ak.to_list(ak.combinations(jetEta,2))
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0):
                leptonsDeltaR = deltaR(leptonsPhi,leptonsEta)
                leptonsDeltaR_N = ak.count(leptonsDeltaR,axis=1)
                leptonsDeltaR = ak.flatten(leptonsDeltaR,axis=1)
            else:
                leptonsDeltaR = []
                leptonsDeltaR_N = ak.zeros_like(leptonsN)
            o2['columns'][sample][year][cat]['LeptonGoodCombinations_deltaR'] = leptonsDeltaR
            o2['columns'][sample][year][cat]['LeptonGoodCombinations_N'] = leptonsDeltaR_N
            
            if(len(jetPhi)!=0 and len(jetEta)!=0):
                jetDeltaR = deltaR(jetPhi,jetEta)
                jetDeltaR_N = ak.count(jetDeltaR,axis=1)
                jetDeltaR = ak.flatten(jetDeltaR,axis=1)
            else:
                jetDeltaR = []
                jetDeltaR_N = ak.zeros_like(jetN)
            o2['columns'][sample][year][cat]['JetGoodCombinations_deltaR'] = jetDeltaR
            o2['columns'][sample][year][cat]['JetGoodCombinations_N'] = jetDeltaR_N
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0 and len(jetPhi)!=0 and len(jetEta)!=0):
                jetLeptonDeltaR = deltaR(jetLeptonPhi,jetLeptonEta)
                jetLeptonDeltaR_N = ak.count(jetLeptonDeltaR,axis=1)
                jetLeptonDeltaR = ak.flatten(jetLeptonDeltaR,axis=1)
            else:
                jetLeptonDeltaR = []
                jetLeptonDeltaR_N = ak.zeros_like(leptonsN)
            o2['columns'][sample][year][cat]['JetLeptonGoodCombinations_deltaR'] = jetLeptonDeltaR
            o2['columns'][sample][year][cat]['JetLeptonGoodCombinations_N'] = jetLeptonDeltaR_N
            
            if(len(leptonsPhi)!=0 and len(leptonsEta)!=0 and len(fatJetPhi)!=0 and len(fatJetEta)!=0):
                fatjetLeptonDeltaR = deltaR(fatjetLeptonPhi,fatjetLeptonEta)
                fatjetLeptonDeltaR_N = ak.count(fatjetLeptonDeltaR,axis=1)
                fatjetLeptonDeltaR = ak.flatten(fatjetLeptonDeltaR,axis=1)
            else:
                fatjetLeptonDeltaR = []
                fatjetLeptonDeltaR_N = ak.zeros_like(leptonsN)
            o2['columns'][sample][year][cat]['FatJetLeptonGoodCombinations_deltaR'] = fatjetLeptonDeltaR
            o2['columns'][sample][year][cat]['FatJetLeptonGoodCombinations_N'] = fatjetLeptonDeltaR_N
            
            if(len(fatJetPhi)!=0 and len(fatJetEta)!=0 and len(jetPhi)!=0 and len(jetEta)!=0):
                jetFatJetDeltaR = deltaR(jetFatJetPhi,jetFatJetEta)
                jetFatJetDeltaR_N = ak.count(jetFatJetDeltaR,axis=1)
                jetFatJetDeltaR = ak.flatten(jetFatJetDeltaR,axis=1)
            else:
                jetFatJetDeltaR = []
                jetFatJetDeltaR_N = ak.zeros_like(leptonsN)
            o2['columns'][sample][year][cat]['JetFatJetGoodCombinations_deltaR'] = jetFatJetDeltaR
            o2['columns'][sample][year][cat]['JetFatJetGoodCombinations_N'] = jetFatJetDeltaR_N

TTbbDiLeptonic__nominal TTbbDiLeptonic_Powheg_2018 baseline
TTbbDiLeptonic__nominal TTbbDiLeptonic_Powheg_2018 ee
TTbbDiLeptonic__nominal TTbbDiLeptonic_Powheg_2018 emu
TTbbDiLeptonic__nominal TTbbDiLeptonic_Powheg_2018 mumu


In [28]:
#User parameters
years = ["2018"]
cats = ["baseline","ee","emu","mumu"]
#cats = ["baseline"]
first_sample = 'TTTo2L2Nu' #any sample which contains all columns requested for plotting
vars = ["LeptonGoodCombinations_deltaR"] #columns to plot, leave empty for all columns
modifier = "all" #which objects to plot per event, options are: 'pt_#' where # goes from 1 to N with 1 being the leading pt object
                  #                                              'max' which plots the highest value of the parameter per event
                  #                                              'all' which plots all values of the parameter per event

dataSamples = []
for sample in o['columns'].keys():
    if 'DATA' in sample:
        dataSamples.append(sample)

for year in years:
    for cat in cats:
        if(not len(vars)):
            vars = o['columns'][first_sample+"__nominal"][getName(first_sample)][cat].keys()
        for var in vars:
            if(("weight" in var) or ("Idx" in var) or ("Ttbar" in var)):
                continue
            print(year, cat, var)
            col = var.split("_")[0]
            quantity = var.split("_")[1]
            all_MC = {}
            all_weight = {}
            for label, samples in parameters_dump['plotting_style']['samples_groups'].items():
                label_data = []
                label_weight = {}
                for sample in samples:
                    weight = {}
                    ttbb_weight = {}
                    name = getName(sample)
                    if(("V2" in var) and ("v7" in name)):
                        data = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][var[:-2]].value)
                    else:
                        data = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][var].value)
                    
                    for weightVar in o['columns'][first_sample+"__nominal"][getName(first_sample)][cat].keys():          
                        if ("weight" in weightVar):
                            weight[weightVar] = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][weightVar].value)
                            weight[weightVar] = weight[weightVar] / o['sum_genweights'][name]
                    A = o['sum_genweights_ttB'][name]
                    #if(sample=="TTTo2L2Nu"):
                    if(False):
                        genTtbarId = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat]["events_genTtbarId"].value) % 100
                        tt_ttb_mask = genTtbarId > 50
                        for key in weight:
                            weight[key] = weight[key][~tt_ttb_mask]
                        if(("V2" in var) and ("v7" in name)):
                            ttbb_data = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat][var[:-2]].value)
                        else:
                            ttbb_data = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat][var].value)
                        genTtbarId = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat]["events_genTtbarId"].value) % 100
                        ttbb_ttb_mask = genTtbarId > 50
                        B = o2['sum_genweights_ttB']['TTbbDiLeptonic_Powheg_2018']
                        for weightVar in o['columns'][first_sample+"__nominal"][getName(first_sample)][cat].keys():
                            if ("weight" in weightVar):
                                ttbb_weight[weightVar] = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat][weightVar].value)
                                ttbb_weight[weightVar] = ttbb_weight[weightVar] / o2['sum_genweights']['TTbbDiLeptonic_Powheg_2018']
                                ttbb_weight[weightVar] = ttbb_weight[weightVar][ttbb_ttb_mask]
                                ttbb_weight[weightVar] = ttbb_weight[weightVar] * (A/B)
                            
                        if(quantity!="N" and col!="events"):
                            colN = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][col+"_N"].value)
                            data = ak.unflatten(data,colN)
                            pt_data = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][col+"_pt"].value)
                            pt_data = ak.unflatten(pt_data,colN)
                            sortIndices = ak.argsort(pt_data,ascending=False)
                            data = data[sortIndices]
                            ttbb_colN = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat][col+"_N"].value)
                            ttbb_data = ak.unflatten(ttbb_data,ttbb_colN)
                            ttbb_pt_data = ak.from_numpy(o2['columns']['TTbbDiLeptonic__nominal']['TTbbDiLeptonic_Powheg_2018'][cat][col+"_pt"].value)
                            ttbb_pt_data = ak.unflatten(ttbb_pt_data,ttbb_colN)
                            sortIndices = ak.argsort(ttbb_pt_data,ascending=False)
                            ttbb_data = ttbb_data[sortIndices]
                            if "pt" in modifier:
                                index = int(modifier.split("_")[1]) - 1
                                ttbb_data = ak.mask(ttbb_data, ak.num(ttbb_data) > 1)[:, index]
                                data = ak.mask(data, ak.num(data) > 0)[:, index]
                            elif modifier=="max":
                                data = ak.max(data,axis=1)
                                ttbb_data = ak.max(ttbb_data,axis=1)
                        data = data[~tt_ttb_mask]
                        ttbb_data = ttbb_data[ttbb_ttb_mask]
                        data = ak.concatenate((data,ttbb_data),axis=0)
                        for key in weight:
                            weight[key] = ak.concatenate((weight[key],ttbb_weight[key]),axis=0)
                    
                    elif(quantity!="N" and col!="events"):
                        colN = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][col+"_N"].value)
                        data = ak.unflatten(data,colN)
                        pt_data = ak.from_numpy(o['columns'][sample+"__nominal"][name][cat][col+"_pt"].value)
                        pt_data = ak.unflatten(pt_data,colN)
                        sortIndices = ak.argsort(pt_data,ascending=False)
                        data = data[sortIndices]
                        if "pt" in modifier:
                            index = int(modifier.split("_")[1]) - 1
                            data = ak.mask(data, ak.num(data) > 0)[:, index]
                        elif modifier=="max":
                            data = ak.max(data,axis=1)
                    for key in weight:
                        weight[key], data = ak.broadcast_arrays(weight[key],data)
                        if(modifier=="all"):
                            weight[key] = ak.flatten(weight[key])
                    if(modifier=="all"): 
                        data = ak.flatten(data)
                    none_mask = ak.is_none(data)
                    data = data[~none_mask]
                    for key in weight:
                        weight[key] = weight[key][~none_mask]
                    inf_mask = np.isinf(data)
                    data = data[~inf_mask]
                    for key in weight:
                        weight[key] = weight[key][~inf_mask]
                    label_data = ak.concatenate((label_data,data),axis=0)
                    for key in weight:
                        if key not in label_weight:
                            label_weight[key] = []
                        label_weight[key] = ak.concatenate((label_weight[key],weight[key]),axis=0)
                all_MC[label] = label_data
                for key in label_weight:
                    if key not in all_weight:
                        all_weight[key] = {}
                    all_weight[key][label] = label_weight[key]
                
            bins, xmin, xmax, xlabel = getPlotParams(var)
            plt.style.use([hep.style.ROOT, {'font.size': 22}])
            f2, a2 = plt.subplots()
            if not ("Gen" in var) and not ("gen" in var):
                fig, (ax, rax) = plt.subplots(2, 1, figsize=[12,12], gridspec_kw={"height_ratios": [3,1]}, sharex=True)
                fig.subplots_adjust(hspace=0.06)
                if(bins==-1):
                    n, bin_edges, patches = ax.hist(list(all_MC.values()),weights=list(all_weight["weight_nominal"].values()),stacked=True,label=list(all_MC.keys()))
                else:
                    n, bin_edges, patches = ax.hist(list(all_MC.values()),weights=list(all_weight["weight_nominal"].values()),stacked=True,label=list(all_MC.keys()),bins=bins,range=(xmin,xmax))
                n_sys = {}
                for key in all_weight:
                    if "Down" in key:
                        continue
                    downString = key[:-2]+"Down"
                    err2_up_total = np.zeros_like(n[-1])
                    err2_down_total = np.zeros_like(n[-1])
                    for label in all_weight[key]:
                        if key=="weight_nominal":
                            mcstat_err2 = np.histogram(all_MC[label], bins=bin_edges, weights=all_weight[key][label]**2)[0]
                            #print(label,np.histogram(all_MC[label], bins=bin_edges, weights=all_weight[key][label])[0][3])
                            err2_up_total += mcstat_err2
                            err2_down_total += mcstat_err2
                        else:
                            hist_n_up, hist_bins_up = np.histogram(all_MC[label],weights=all_weight[key][label],bins=bin_edges)
                            hist_n_down, hist_bins_down = np.histogram(all_MC[label],weights=all_weight[downString][label],bins=bin_edges)
                            hist_n, hist_bins = np.histogram(all_MC[label],weights=all_weight["weight_nominal"][label],bins=bin_edges)
                            err_up = hist_n_up - hist_n
                            err_down = hist_n_down - hist_n
                            # Compute the flags to check which of the two variations (up and down) are pushing the nominal value up and down
                            up_is_up = err_up > 0
                            down_is_down = err_down < 0
                            # Compute the flag to check if the uncertainty is one-sided, i.e. when both variations are up or down
                            is_onesided = up_is_up ^ down_is_down
                            # Sum in quadrature of the systematic uncertainties taking into account if the uncertainty is one- or double-sided
                            err2_up_twosided = np.where(up_is_up, err_up**2, err_down**2)
                            err2_down_twosided = np.where(up_is_up, err_down**2, err_up**2)
                            err2_max = np.maximum(err2_up_twosided, err2_down_twosided)
                            err2_up_onesided = np.where(is_onesided & up_is_up, err2_max, 0)
                            err2_down_onesided = np.where(is_onesided & down_is_down, err2_max, 0)
                            err2_up_combined = np.where(is_onesided, err2_up_onesided, err2_up_twosided)
                            err2_down_combined = np.where(
                                is_onesided, err2_down_onesided, err2_down_twosided
                            )
                            err2_up_total += err2_up_combined
                            err2_down_total += err2_down_combined
                    n_sys[key] = err2_up_total
                    n_sys[downString] = err2_down_total
                ax.tick_params(axis='x', labelsize=22)
                ax.tick_params(axis='y', labelsize=22)
                ax.set_xlim(bin_edges[0],bin_edges[-1])
                #systematic error band
                err2_up = np.zeros_like(n[-1])
                err2_down = np.zeros_like(n[-1])
                for key in n_sys:
                    if "Down" in key:
                        continue
                    downString = key[:-2]+"Down"
                    err2_up += n_sys[key]
                    err2_down += n_sys[downString]
                up = n[-1] + np.sqrt(err2_up)
                down = n[-1] - np.sqrt(err2_down)
                ratio_up = np.where(n[-1] != 0, up / n[-1], 1)
                ratio_down = np.where(n[-1] != 0, down / n[-1], 1)
                unc_band = np.array([ratio_down, ratio_up])
                #print("ratio_up",ratio_up)
                #print("ratio_down",ratio_down)
                rax.fill_between(
                    bin_edges,
                    np.r_[unc_band[0], unc_band[0, -1]],
                    np.r_[unc_band[1], unc_band[1, -1]],
                    label="syst. unc.",
                    color= [0.,0.,0.,0.4],
                    facecolor= [0.,0.,0.,0.],
                    hatch= "////",
                    linewidth= 0,
                    step= 'post',
                    zorder= 2
                )
                rax.hlines(
                    1.0, *bin_edges[[0, -1]], colors='gray', linestyles='dashed'
                )
                
                ax.legend(ncol=2,loc="upper right",fontsize=16)
                plt.xlabel(xlabel,fontsize=16)
                nData = [0.0]*len(bin_edges[:-1])
                for sample in dataSamples:
                    for name in o['columns'][sample].keys():
                        data = ak.from_numpy(o['columns'][sample][name][cat][var].value)
                        if(quantity!="N" and col!="events"):
                            colN = ak.from_numpy(o['columns'][sample][name][cat][col+"_N"].value)
                            data = ak.unflatten(data,colN)  
                            pt_data = ak.from_numpy(o['columns'][sample][name][cat][col+"_pt"].value)
                            pt_data = ak.unflatten(pt_data,colN)
                            sortIndices = ak.argsort(pt_data,ascending=False)
                            data = data[sortIndices]
                            if "pt" in modifier:
                                index = int(modifier.split("_")[1]) - 1
                                data = ak.mask(data, ak.num(data) > 0)[:, index]
                            elif modifier=="max":
                                data = ak.max(data,axis=1)
                        none_mask = ak.is_none(data)
                        data = data[~none_mask]
                        inf_mask = np.isinf(data)
                        data = data[~inf_mask]
                        if(modifier=="all"):
                            data = ak.flatten(data)
                        n_data, bin_edges_data, patches_data = a2.hist(data,bins=bin_edges)
                        nData += n_data
                error = np.sqrt(nData)
                binCenters = (bin_edges_data[:-1] + bin_edges_data[1:]) / 2
                ratio = nData/n[-1]
                ratio_error = error/n[-1]
                ratio_error[np.isnan(ratio_error)] = np.inf
                #print("data sum",nData[28])
                #print("mc sum",n[-1])
                #print("ratio",ratio)
                #print("ratio_error",ratio_error)
                ax.errorbar(binCenters, nData, yerr=error,linestyle='solid',markersize=5.0,color='black',elinewidth=1,linewidth=0,marker='.')
                rax.errorbar(binCenters, ratio, yerr=ratio_error,linestyle='solid',markersize=5.0,color='black',elinewidth=1,linewidth=0,marker='.')
                rax.set_ylim((0.5, 1.5))
                rax.yaxis.set_label_coords(-0.075, 1)
                rax.tick_params(axis='x', labelsize=22)
                rax.tick_params(axis='y', labelsize=22)
                max_data = max(nData)
                max_MC = max(n[-1])
                if(max_data>max_MC):
                    ax.set_ylim((0, 2.0 * max_data))
                else:
                    ax.set_ylim((0, 2.0 * max_MC))
            else:
                f1, a1 = plt.subplots()
                if(bins==-1):
                    n, bin_edges, patches = a1.hist(list(all_MC.values()),weights=list(all_weight["weight_nominal"].values()),stacked=True,label=list(all_MC.keys()))
                else:
                    n, bin_edges, patches = a1.hist(list(all_MC.values()),weights=list(all_weight["weight_nominal"].values()),stacked=True,label=list(all_MC.keys()),bins=bins,range=(xmin,xmax))
                ax.legend(ncol=2,loc="upper right")
            
            plt.close(f2)
            if "pt" in modifier:
                stringMod = modifier.split("_")[1]
            filepath = f"hists/{year}/{cat}/{col}"
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            plt.savefig(filepath+"/"+var+"_"+stringMod+".pdf")
            plt.show()
            plt.clf()
            plt.cla()
            plt.close()

2018 baseline LeptonGoodCombinations_deltaR


AttributeError: no field named 'value'

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/awkward/highlevel.py#L1124)

In [50]:
for sample in o['columns'].keys():
    if "DATA" in sample:
         continue
    for year in o['columns'][sample].keys():
        for cat in o['columns'][sample][year].keys():
            print(sample,year,cat)
            weights = ak.from_numpy(o['columns'][sample][year][cat]['weight_nominal'].value) * 1/o['sum_genweights'][year]
            print(type(o['columns'][sample][year][cat]['weight_nominal'].value))
            print(weights)
            print(ak.sum(weights,axis=0))a
            print(o['sumw'][cat][year][sample[:-9]])
            print("cutflow",o['cutflow'][cat][year][sample[:-9]])
            #print("num events",len(ak.from_numpy(o['columns'][sample][year][cat]['BJetGood_N'].value)))

ZJetsToQQ_HT600to800__nominal ZJetsToQQ_HT600to800_v7__2018 baseline
<class 'numpy.ndarray'>
[0.126, 0.207, 0.11, 0.128, 0.115]
0.686193402916419
0.6861934029164191
cutflow 5
ZJetsToQQ_HT600to800__nominal ZJetsToQQ_HT600to800_v7__2018 ee
<class 'numpy.ndarray'>
[]
0.0
0.0
cutflow 0
ZJetsToQQ_HT600to800__nominal ZJetsToQQ_HT600to800_v7__2018 emu
<class 'numpy.ndarray'>
[0.128]
0.1277717789400299
0.1277717789400299
cutflow 1
ZJetsToQQ_HT600to800__nominal ZJetsToQQ_HT600to800_v7__2018 mumu
<class 'numpy.ndarray'>
[0.126, 0.207, 0.11, 0.115]
0.5584216239763891
0.5584216239763891
cutflow 4
ZJetsToQQ_HT400to600__nominal ZJetsToQQ_HT400to600_v7__2018 baseline
<class 'numpy.ndarray'>
[1.29, 0.457]
1.7495743798889176
1.7495743798889176
cutflow 2
ZJetsToQQ_HT400to600__nominal ZJetsToQQ_HT400to600_v7__2018 ee
<class 'numpy.ndarray'>
[]
0.0
0.0
cutflow 0
ZJetsToQQ_HT400to600__nominal ZJetsToQQ_HT400to600_v7__2018 emu
<class 'numpy.ndarray'>
[0.457]
0.45723432393972385
0.45723432393972385
cutflow 1

In [6]:
#o3 = load("out_columns/output_newconfig14_all.coffea")
o3 = load("out_columns/output_allNoDupes.coffea")

In [7]:
for sample in o3['columns'].keys():
    if "DATA" in sample:
         continue
    for year in o3['columns'][sample].keys():
        for cat in o3['columns'][sample][year].keys():
            print(sample,year,cat)
            weights = ak.from_numpy(o3['columns'][sample][year][cat]['weight_nominal'].value) * 1/o3['sum_genweights'][year]
            print(type(o3['columns'][sample][year][cat]['weight_nominal'].value))
            print(weights)
            print(ak.sum(weights,axis=0))
            print(o3['sumw'][cat][year][sample[:-9]])
            print("cutflow",o3['cutflow'][cat][year][sample[:-9]])
            #print("num events",len(ak.from_numpy(o['columns'][sample][year][cat]['BJetGood_N'].value)))

QCD_HT500to700__nominal QCD_HT500to700_v7__2018 baseline
<class 'numpy.ndarray'>
[35.9, 33, 35.1, 25.7, 27.8]
157.53687013933532
157.53687013933532
cutflow 5
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 ee
<class 'numpy.ndarray'>
[35.1, 27.8]
62.86387541249354
62.863875412493535
cutflow 2
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 emu
<class 'numpy.ndarray'>
[35.9]
35.92598716212154
35.92598716212154
cutflow 1
QCD_HT500to700__nominal QCD_HT500to700_v7__2018 mumu
<class 'numpy.ndarray'>
[33, 25.7]
58.74700756472024
58.74700756472024
cutflow 2
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 baseline
<class 'numpy.ndarray'>
[0.366, 0.453, 0.573, 0.41, 0.32, 0.368, ... 0.576, 0.347, 0.355, 0.528, 0.401]
309.4470876132873
309.44708761328724
cutflow 682
WJetsToLNu_HT-400To600__nominal WJetsToLNu_HT-400To600__2018 ee
<class 'numpy.ndarray'>
[0.573, 0.41, 0.32, 0.368, 0.346, 0.363, ... 0.513, 0.39, 0.508, 0.487, 0.576, 0.355]
107.888569889384
107.88856988938404
cutflow 244

In [16]:
sample_dict = {}
for num in [1,2,3,4,5,6,7,8,9,11,12,15,16,17,18,19,20,-1,-2,-3]:
    print("checking file num",num)
    if(num>0):
        o3 = load(f"out_columns/output_newconfig{num}_all.coffea")
    elif num==-1:
        o3 = load("/afs/cern.ch/user/a/asparker/public/dec_ttH/PocketCoffea/AnalysisConfigs/configs/ttHbb/jan24_all/output_newconfigLost2_all.coffea")
    elif num==-2:
        o3 = load("/afs/cern.ch/user/a/asparker/public/dec_ttH/PocketCoffea/AnalysisConfigs/configs/ttHbb/jan24_all/output_newconfigLost1_all.coffea")
    elif num==-3:
        o3 = load("/afs/cern.ch/user/a/asparker/public/dec_ttH/PocketCoffea/AnalysisConfigs/configs/ttHbb/jan24_all_test_new_3b/output_newconfigLost3_all.coffea")
    for sample in o3['columns'].keys():
        if sample not in sample_dict:
            sample_dict[sample] = num
        else:
            print("duplicate sample",sample,"original file num",sample_dict[sample])

checking file num 1
checking file num 2
checking file num 3
checking file num 4
checking file num 5
checking file num 6
checking file num 7
checking file num 8
checking file num 9
checking file num 11
checking file num 12
checking file num 15
checking file num 16
checking file num 17
checking file num 18
checking file num 19
checking file num 20
checking file num -1
checking file num -2
checking file num -3


In [14]:
print(o['cutflow']['baseline']['ttHTobb_2018']['ttHTobb'])
print(o['sum_genweights']['ttHTobb_2018'])
print(o['columns']['ttHTobb__nominal']['ttHTobb_2018']['baseline']['weight_nominal'].value)
weights = ak.from_numpy(o['columns']['ttHTobb__nominal']['ttHTobb_2018']['baseline']['weight_nominal'].value)
weights = weights * (1/o['sum_genweights']['ttHTobb_2018'])
weights = np.square(weights)
print("weights",weights)
print('sum_genweights',o['sum_genweights']['ttHTobb_2018'])
weights = ak.sum(weights,axis=0)
print("weight sum",weights)
weights = np.sqrt(weights)
print(weights)

49427
4843452.0
[10320.98820507 11759.84001833  8793.69339794 ...  5480.94545939
  7263.42749791  6984.99281966]
weights [4.54e-06, 5.9e-06, 3.3e-06, 4.67e-06, ... 1.54e-06, 1.28e-06, 2.25e-06, 2.08e-06]
sum_genweights 4843452.0
weight sum 0.1648807108934491
0.4060550589433028


In [26]:
o3 = load("output_config_general_TTbbSemiLeptonic.coffea")
print(o3['cutflows'].keys())

KeyError: 'cutflows'